<a href="https://colab.research.google.com/github/NiharikaVadlamudi/BoundaryNet/blob/main/ASR_GLUE_PIPELINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the Google Drive 
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!pip install vit_pytorch
!pip install empatches

In [ ]:
!pip install plantcv
from plantcv import plantcv as pcv
pcv.params.debug = None 

In [ ]:
import copy
import cv2
import os 
import pathlib
import sys 
import numpy as np 
import numpy as np
import matplotlib.pyplot as plt
from vit_pytorch import ViT
from einops import rearrange
from empatches import EMPatches


import torch
from torch import nn
import torch.nn.functional as F
from vit_pytorch.vit import Transformer

import math
from shapely import geometry
from skimage.graph import MCP_Connect
from skimage.morphology import skeletonize
from skimage.measure import label as skimage_label
from sklearn.metrics.pairwise import euclidean_distances
from scipy.signal import convolve2d
from collections import defaultdict

from scipy.spatial import distance
import itertools
import numba
import itertools
import glob
import csv 
import json 
from collections import OrderedDict



from google.colab.patches import cv2_imshow


In [ ]:
import argparse
import json
from itertools import combinations

from shapely.affinity import scale
from shapely.geometry import LineString, MultiPolygon, Polygon


import json 
import os 
import cv2 
import numpy as np 
from google.colab.patches import cv2_imshow

In [ ]:
THRESHOLD = 0.4 ## binarization threshold after the model output
SPLITSIZE =  256  ## your image will be divided into patches of 256x256 pixels
setting = "base"  ## choose the desired model size [small, base or large], depending on the model you want to use
patch_size = 8 ## choose your desired patch size [8 or 16], depending on the model you want to use
image_size =  (SPLITSIZE,SPLITSIZE)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if setting == 'base':
    encoder_layers = 6
    encoder_heads = 8
    encoder_dim = 768

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

IMAGE_PATH='/content/_data_ASR_Images_ASR-B02-S10_B02-S10-191.jpg'
BMODEL = '/content/drive/MyDrive/researchTesting/best-model_8_2018base_256_8.pt'
SMODEL = '/content/drive/MyDrive/researchTesting/try.pt'

In [ ]:


class ScribbleNet(nn.Module):
    """
    The autoencoder model to enhance images in an image to image translation fashion.
    This code is built on top of the vit-pytorch code https://github.com/lucidrains/vit-pytorch.
    Args:
        encoder (model): the defined encoder, hete it is a ViT
        decoder_dim (int): decoder dim (embedding size)
        decoder_depth (int): number of decoder layers
        decoder_heads (int): number of decoder heads
        decoder_dim_head (int): decoder head dimension
    """
    def __init__(
        self,
        *,
        encoder,
        decoder_dim,
        decoder_depth = 1,
        decoder_heads = 8,
        decoder_dim_head = 64
    ):
        super().__init__()
        # extract hyperparameters and functions from the ViT encoder.
        self.encoder = encoder
        num_patches, encoder_dim = encoder.pos_embedding.shape[-2:]
        self.to_patch, self.patch_to_emb = encoder.to_patch_embedding[:2]
        pixel_values_per_patch = self.patch_to_emb.weight.shape[-1]

        # define your decoder here
        self.enc_to_dec = nn.Linear(encoder_dim, decoder_dim) if encoder_dim != decoder_dim else nn.Identity()
        self.mask_token = nn.Parameter(torch.randn(decoder_dim))
        self.decoder = Transformer(dim = decoder_dim, depth = decoder_depth, heads = decoder_heads, dim_head = decoder_dim_head, mlp_dim = decoder_dim * 4)
        self.decoder_pos_emb = nn.Embedding(num_patches, decoder_dim)
        self.to_pixels = nn.Linear(decoder_dim, pixel_values_per_patch)

    def forward(self, img, gt_img):
        
        # get patches and their number
        patches = self.to_patch(img)
        _, num_patches, *_ = patches.shape

        # project pixel patches to tokens and add positions
        tokens = self.patch_to_emb(patches)
        tokens = tokens + self.encoder.pos_embedding[:, 1:(num_patches + 1)]

        # encode tokens by the encoder
        encoded_tokens = self.encoder.transformer(tokens)

        # project encoder to decoder dimensions, if they are not equal.
        decoder_tokens = self.enc_to_dec(encoded_tokens)

        # decode tokens with decoder
        decoded_tokens = self.decoder(decoder_tokens)

        # project tokens to pixels
        pred_pixel_values = self.to_pixels(decoded_tokens)

        # calculate the loss with gt
        gt_patches = self.to_patch(gt_img)
        loss = F.mse_loss(pred_pixel_values, gt_patches)

        return loss, patches, pred_pixel_values

In [ ]:
# Instantiate twice and load 2 different weight checkpoints 

v1 = ViT(
    image_size = image_size,
    patch_size = patch_size,
    num_classes = 1000,
    dim = encoder_dim,
    depth = encoder_layers,
    heads = encoder_heads,
    mlp_dim = 2048
)

v2 = ViT(
    image_size = image_size,
    patch_size = patch_size,
    num_classes = 1000,
    dim = encoder_dim,
    depth = encoder_layers,
    heads = encoder_heads,
    mlp_dim = 2048
)

# Binary Output 
binaryModel = ScribbleNet(
    encoder = v1,
    decoder_dim = encoder_dim,      
    decoder_depth = encoder_layers,
    decoder_heads = encoder_heads       
)

# Scribble Output 
scribbleModel = ScribbleNet(
    encoder = v2,
    decoder_dim = encoder_dim,      
    decoder_depth = encoder_layers,
    decoder_heads = encoder_heads       
)

# Sending model to the device..
scribbleModel=scribbleModel.to(device)
binaryModel=binaryModel.to(device)

# Model Weight Loadings ...
binaryModel.load_state_dict(torch.load(BMODEL, map_location=device),strict=False)
scribbleModel.load_state_dict(torch.load(SMODEL, map_location=device),strict=False)


In [ ]:
# Utils for output cleaning 
def preprocess(deg_img):
    deg_img = (np.array(deg_img) /255).astype('float32')
    # normalize data
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    out_deg_img = np.zeros([3, *deg_img.shape[:-1]])
    for i in range(3):
        out_deg_img[i] = (deg_img[:,:,i] - mean[i]) / std[i]
    return out_deg_img

def readFullImage(path,PDIM=256,DIM=256,OVERLAP=0.25):    
    input_patches=[]
    emp = EMPatches()
    try:
        img = cv2.imread(path)
        img = preprocess(img)
        img = np.transpose(img)
        img_patches, indices = emp.extract_patches(img,patchsize=PDIM,overlap=OVERLAP)
        for i,patch in enumerate(img_patches):
            resized=[DIM,DIM]
            if patch.shape[0]!= DIM or patch.shape[1]!= DIM : 
                resized=[patch.shape[0],patch.shape[1]]
                patch = cv2.resize(patch,(DIM,DIM),interpolation = cv2.INTER_AREA)
            patch = np.asarray(patch,dtype=np.float32)
            patch =  np.transpose(patch) 
            patch= np.expand_dims(patch,axis=0)
            sample={'img':patch,'resized':resized}
            input_patches.append(sample)
    except Exception as exp : 
        print('ImageReading Error ! :{}'.format(exp))
        return None 
    return input_patches,indices



def inferenceTestImage(scribbleModel,binaryModel,path,PDIM=256,DIM=256,OVERLAP=0.25,save=True):
    emp = EMPatches()
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    if not os.path.exists(path):
        print('Invalid File Path !')
        sys.exit()
    else:
        res_patches , indices = readFullImage(path,PDIM,DIM,OVERLAP)
        
        scribble_output_patches=[]
        binary_output_patches=[]

        for i,sample in enumerate(res_patches):
            p = sample['img']
            target_shape = (sample['resized'][1],sample['resized'][0])

            with torch.no_grad():
                inputs =torch.from_numpy(p).to(device)

                # Pass through SModel 
                _,_,spred_pixel_values = scribbleModel(inputs,inputs)

                # Pass through BModel 
                _,_,bpred_pixel_values = binaryModel(inputs,inputs)

                # Scribble Pixels 
                srec_patches = spred_pixel_values
                brec_patches = bpred_pixel_values

                soutput_image = rearrange(srec_patches, 'b (h w) (p1 p2 c) -> b c (h p1) (w p2)',p1 = patch_size, p2 = patch_size,  h=image_size[0]//patch_size)
                boutput_image = rearrange(brec_patches, 'b (h w) (p1 p2 c) -> b c (h p1) (w p2)',p1 = patch_size, p2 = patch_size,  h=image_size[0]//patch_size)
                
                soutput_image = soutput_image.cpu().numpy().squeeze()
                boutput_image = boutput_image.cpu().numpy().squeeze()

                soutput_image =  np.transpose(soutput_image,(2, 1, 0))
                boutput_image =  np.transpose(boutput_image,(2, 1, 0))

                # Resizing to get desired output 
                boutput_image = cv2.resize(boutput_image,target_shape, interpolation = cv2.INTER_AREA)
                soutput_image = cv2.resize(soutput_image,target_shape, interpolation = cv2.INTER_AREA)

                for ch in range(3):
                    soutput_image[:,:,ch] = (soutput_image[:,:,ch] *std[ch]) + mean[ch]
                    boutput_image[:,:,ch] = (boutput_image[:,:,ch] *std[ch]) + mean[ch]
                

                # Basic Thresholding 
                soutput_image[np.where(soutput_image>1)] = 1
                soutput_image[np.where(soutput_image<0)] = 0

                boutput_image[np.where(boutput_image>1)] = 1
                boutput_image[np.where(boutput_image<0)] = 0


                # binarize the predicted image taking 0.5 as threshold
                soutput_image = (soutput_image>THRESHOLD)*1
                boutput_image = (boutput_image>THRESHOLD)*1

                # Append the net processed patch 
                scribble_output_patches.append(255*soutput_image)
                binary_output_patches.append(255*boutput_image)
                


        assert len(res_patches) == len(scribble_output_patches)==len(binary_output_patches)," Missing patches while processing !"
      
        
        # Restich the image 
        scribbleImage = emp.merge_patches(scribble_output_patches,indices)
        binaryImage = emp.merge_patches(binary_output_patches,indices)

        scribbleImage = np.transpose(scribbleImage,(1,0,2))
        binaryImage = np.transpose(binaryImage,(1,0,2))

        return scribbleImage,binaryImage

In [ ]:
def split_polygons(polygons: list, eps) -> list:
    """
    Split the touching and overlapping polygons.
    :param polygons: The polygons to split.
    :return polygons: The non-touching polygons.
    """
    # Convert them into Shapely's polygon 
    polygons = [Polygon(poly) for poly in polygons if len(poly)>=4]

    for comb in combinations(range(len(polygons)), 2):
        poly1 = polygons[comb[0]]
        poly2 = polygons[comb[1]]
        # Skip invalid polygons as they cannot be compared.
        if not poly1.is_valid or not poly2.is_valid:
            continue
        # If the two polygons intersect: first erode them, then check if they still intersect.
        if poly1.intersects(poly2):
            poly1 = poly1.buffer(-eps)
            poly2 = poly2.buffer(-eps)
            intersection = poly1.intersection(poly2)
            # If they still intersect, remove the intersection from the biggest polygon.
            if not intersection.is_empty:
                if (
                    isinstance(intersection, Polygon)
                    and intersection.area < 0.2 * poly1.area
                    and intersection.area < 0.2 * poly2.area
                    or isinstance(intersection, MultiPolygon)
                ):
                    if poly1.area > poly2.area:
                        polygons[comb[0]] = poly1.difference(intersection)
                        polygons[comb[1]] = poly2.buffer(-eps)
                    else:
                        polygons[comb[1]] = poly2.difference(intersection)
                        polygons[comb[0]] = poly1.buffer(-eps)
                elif isinstance(intersection, LineString):
                    polygons[comb[0]] = poly1.difference(intersection)
                    polygons[comb[1]] = poly2.difference(intersection)
        elif poly1.touches(poly2):
            polygons[comb[0]] = poly1.buffer(-2 * eps)
            polygons[comb[1]] = poly2.buffer(-2 * eps)
    # Erode all polygons so that they don't touch when drawn over the label image.
    polygons = [poly.buffer(-2 * eps) for poly in polygons]
    return polygons

def int_coords(x: np.array) -> np.array:
  return np.array(x).round().astype(np.int32)

def generateNewPolygons(image,polygons,eps=2,vis=True):
  newpolys = split_polygons(polygons,eps = eps)
  # Convert them to points 
  gdpolygons = []
  for p in newpolys:
    poly = int_coords(p.exterior.coords)
    gdpolygons.append(poly.tolist())

  if vis : 
    newCanvas = np.zeros(image.shape)
    for element in gdpolygons : 
      npoly = np.asarray(element,dtype=np.int32).reshape((-1,1,2))
      newCanvas  = cv2.fillPoly(newCanvas , [npoly], color=(255,255,255))
  return gdpolygons,newCanvas      

In [ ]:
x# Call the pipeline to be ready 

s,b = inferenceTestImage(scribbleModel,binaryModel,IMAGE_PATH,PDIM=256,DIM=256,OVERLAP=0.25,save=True)
# _,b = inferenceTestImage(scribbleModel,binaryModel,IMAGE_PATH,PDIM=256,DIM=256,OVERLAP=0.50,save=True)
print('Scribble Output : ')
cv2_imshow(s)
print('Binary Output : ')
cv2_imshow(b)
cv2.imwrite('bin_191.jpg',b)

In [ ]:
# New Utils 
def deformat(listofpoints):
    # Input : [[[x1,y1],[[x2,y2]],[[x3,y3]]....]
    # Output : [ [x1,y1], [x2,y2],[x3,y3]....]
    output = [ pt[0].tolist() for pt in listofpoints ]
    return output 

def oBox(c):
  points = np.array(c)
  # Fill the bbox 
  rect = cv2.minAreaRect(points)
  box = cv2.boxPoints(rect)
  box = np.int0(box)
  return box 

# Supply the raw image here
def cleanImageFindContours(patch,threshold = 0.10):
  try:
    patch = np.uint8(patch)
    patch = cv2.cvtColor(patch,cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(patch,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours)<1:
      print('No contours in the raw image!')
      return patch
    # Else sort them 
    cntsSorted = sorted(contours, key=lambda x: cv2.contourArea(x),reverse=True)
    areaList = [cv2.contourArea(c) for c in cntsSorted]
    maxArea = max(areaList)
    sortedContours = [deformat(c) for c in cntsSorted if cv2.contourArea(c)>threshold*maxArea]
    return sortedContours

  except Exception as exp : 
    print('Error in figuring out the clean contours ')
    return None 

In [ ]:
def firstProcess(s):
  # Get the clean polygons first 
  contours = cleanImageFindContours(s.copy(),threshold = 0.05)
  patch = np.uint8(s.copy())
  print('Old Eroded Canvas')
  newcanvas = np.zeros(s.shape)
  newobboxs = []
  for c in contours :
    c = np.asarray(c)
    box =  oBox(c)
    newobboxs.append(box)
    newcanvas = cv2.fillPoly(newcanvas,pts=[box],color=(255,255,255))
  cv2_imshow(newcanvas)
  # Send it to the canvas 
  polys,canv=generateNewPolygons(newcanvas,newobboxs,eps=2,vis=True)
  print('NEW Canvas')
  cv2_imshow(canv)
  return canv,polys

snew,polys = firstProcess(s)

In [ ]:
# Skeletonise 

def scribbleGeneration(img,box):
    h,w,c = img.shape
    canvas = np.zeros((h,w))
    box = np.asarray(box,dtype=np.int32)
    box = box.reshape(-1,1,2)
    canvas = cv2.fillPoly(canvas,pts=np.int32([box]), color=(255,255,255))
    skeleton = pcv.morphology.skeletonize(canvas)
    pruned_skeleton, segmented_img, segment_objects = pcv.morphology.prune(skel_img=skeleton,size=100)
    # cv2_imshow(pruned_skeleton)
    scribble = np.asarray(segment_objects[0]).tolist()
    return scribble 



In [ ]:
img = cv2.imread(IMAGE_PATH)
nc = np.zeros(img.shape)
scribbles=[]
for i,pol in enumerate(polys):
  s = scribbleGeneration(img,pol)
  scribbles.append(s)

In [ ]:
parentCanvas = np.zeros(img.shape)
for c in scribbles:
  c = np.asarray(c)
  parentCanvas = cv2.polylines(parentCanvas,[c],False,(255,255,255),3)
cv2_imshow(parentCanvas)

In [ ]:
def findClosePointsV2(p1,p2,dist):
  hdist = np.abs(p1[1]-p2[1])**2
  vdist = np.abs(p1[0]-p2[0])**2
  pointDist = np.sqrt(hdist+vdist)
  if pointDist<=dist :
    return True
  else:
    return False

def extremePoints(c):
    c = np.asarray(c).reshape(-1,1,2)
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    return extLeft,extRight

def fuseScribbles(scribbles,patch,distanceThreshold=100):
    # Collect all the startPoints , endPoints ( with their parent index )
    endPointDict = []
    startPointDict = []
    for i in range(0,len(scribbles)):
      # Grab the start and end points.
      endPoint,startPoint = extremePoints(scribbles[i])
      endPointDict.append(list(endPoint))
      startPointDict.append(list(startPoint))
    
    # Draw the connecting scribbles on the patch.
    gapCanvas = np.zeros(patch.shape)
    for i in range(0,len(startPointDict)):
      for j in range(0,len(endPointDict)):
          if i!=j:
            if j>i :
              decision = findClosePointsV2(startPointDict[i],endPointDict[j],distanceThreshold)
              if decision: 
                  gapCanvas = cv2.line(gapCanvas,startPointDict[i],endPointDict[j],color=(255,255,255),thickness=3)

            else:
              decision = findClosePointsV2(endPointDict[i],startPointDict[j],distanceThreshold)
              if decision: 
                  gapCanvas = cv2.line(gapCanvas,endPointDict[i],startPointDict[j],color=(255,255,255),thickness=3)
          else:
            continue 
    
    # Return this canvas 
    return gapCanvas

In [ ]:
gapMap = fuseScribbles(scribbles,img)
cv2_imshow(gapMap)

In [ ]:
netMap = cv2.bitwise_or(parentCanvas,gapMap)
finalContours = cleanImageFindContours(netMap,threshold=0.1)
# # Apply find contours on this one 
fcanvas = np.zeros(netMap.shape)
for c in finalContours:
  c = np.asarray(c)
  fcanvas = cv2.polylines(fcanvas,[c],False,(255,255,255),5)

cv2_imshow(fcanvas)

In [ ]:

'''
New Additional Utils : V2 
'''

import cv2 as cv

def houghLinesProcessing(src,vis=False):

    # Grayscale + Thresholding
    src = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
    src[src > 127] = 255
    src[src <= 127] = 0
    
    
    dst = np.copy(src)
    # Copy edges to the images that will display the results in BGR
    cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)
    cdstP = np.copy(cdst)

    dst = np.uint8(dst)
    lines = cv.HoughLines(dst, 1, np.pi / 180, 150, None, 0, 0)

    if vis : 
        if lines is not None:
            for i in range(0, len(lines)):
                rho = lines[i][0][0]
                theta = lines[i][0][1]
                a = math.cos(theta)
                b = math.sin(theta)
                x0 = a * rho
                y0 = b * rho
                pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
                pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
                cv.line(cdst, pt1, pt2, (0,0,255), 3, cv.LINE_AA)

    # DST MAP
    linesP = cv.HoughLinesP(dst, 1, np.pi / 180, 50, None, 50, 10)

    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            cv.line(cdstP, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv.LINE_AA)

    return linesP,cdstP
   
class HoughBundler:     
    def __init__(self,min_distance=5,min_angle=2):
        self.min_distance = min_distance
        self.min_angle = min_angle
    
    def get_orientation(self, line):
        orientation = math.atan2(abs((line[3] - line[1])), abs((line[2] - line[0])))
        return math.degrees(orientation)

    def check_is_line_different(self, line_1, groups, min_distance_to_merge, min_angle_to_merge):
        for group in groups:
            for line_2 in group:
                if self.get_distance(line_2, line_1) < min_distance_to_merge:
                    orientation_1 = self.get_orientation(line_1)
                    orientation_2 = self.get_orientation(line_2)
                    if abs(orientation_1 - orientation_2) < min_angle_to_merge:
                        group.append(line_1)
                        return False
        return True

    def distance_point_to_line(self, point, line):
        px, py = point
        x1, y1, x2, y2 = line

        def line_magnitude(x1, y1, x2, y2):
            line_magnitude = math.sqrt(math.pow((x2 - x1), 2) + math.pow((y2 - y1), 2))
            return line_magnitude

        lmag = line_magnitude(x1, y1, x2, y2)
        if lmag < 0.00000001:
            distance_point_to_line = 9999
            return distance_point_to_line

        u1 = (((px - x1) * (x2 - x1)) + ((py - y1) * (y2 - y1)))
        u = u1 / (lmag * lmag)

        if (u < 0.00001) or (u > 1):
            #// closest point does not fall within the line segment, take the shorter distance
            #// to an endpoint
            ix = line_magnitude(px, py, x1, y1)
            iy = line_magnitude(px, py, x2, y2)
            if ix > iy:
                distance_point_to_line = iy
            else:
                distance_point_to_line = ix
        else:
            # Intersecting point is on the line, use the formula
            ix = x1 + u * (x2 - x1)
            iy = y1 + u * (y2 - y1)
            distance_point_to_line = line_magnitude(px, py, ix, iy)

        return distance_point_to_line

    def get_distance(self, a_line, b_line):
        dist1 = self.distance_point_to_line(a_line[:2], b_line)
        dist2 = self.distance_point_to_line(a_line[2:], b_line)
        dist3 = self.distance_point_to_line(b_line[:2], a_line)
        dist4 = self.distance_point_to_line(b_line[2:], a_line)

        return min(dist1, dist2, dist3, dist4)

    def merge_lines_into_groups(self, lines):
        groups = []  # all lines groups are here
        # first line will create new group every time
        groups.append([lines[0]])
        # if line is different from existing gropus, create a new group
        for line_new in lines[1:]:
            if self.check_is_line_different(line_new, groups, self.min_distance, self.min_angle):
                groups.append([line_new])

        return groups

    def merge_line_segments(self, lines):
        orientation = self.get_orientation(lines[0])
      
        if(len(lines) == 1):
            return np.block([[lines[0][:2], lines[0][2:]]])

        points = []
        for line in lines:
            points.append(line[:2])
            points.append(line[2:])
        if 45 < orientation <= 90:
            #sort by y
            points = sorted(points, key=lambda point: point[1])
        else:
            #sort by x
            points = sorted(points, key=lambda point: point[0])

        return np.block([[points[0],points[-1]]])

    def process_lines(self, lines):
        lines_horizontal  = []
        lines_vertical  = []
  
        for line_i in [l[0] for l in lines]:
            orientation = self.get_orientation(line_i)
            # if vertical
            if 45 < orientation <= 90:
                lines_vertical.append(line_i)
            else:
                lines_horizontal.append(line_i)

        lines_vertical  = sorted(lines_vertical , key=lambda line: line[1])
        lines_horizontal  = sorted(lines_horizontal , key=lambda line: line[0])
        merged_lines_all = []

        # for each cluster in vertical and horizantal lines leave only one line
        for i in [lines_horizontal, lines_vertical]:
            if len(i) > 0:
                groups = self.merge_lines_into_groups(i)
                merged_lines = []
                for group in groups:
                    merged_lines.append(self.merge_line_segments(group))
                merged_lines_all.extend(merged_lines)
                    
        return np.asarray(merged_lines_all)

In [ ]:

# Apply the global post processing - Hough Lines 
reImage = copy.deepcopy(parentCanvas)
reImage = np.asarray(reImage)
print('REIMAGE OLD : {}'.format(reImage.shape))
res = np.transpose(reImage)
print('REIMAGE NEW : {}'.format(res.shape))
bundler = HoughBundler(min_distance=15,min_angle=5)


In [ ]:
# Apply the global post processing - Hough Lines 
reImage = copy.deepcopy(fcanvas.astype(np.uint8))
bundler = HoughBundler(min_distance=15,min_angle=5)
linesP,hough_canvas=houghLinesProcessing(reImage,vis=False)
filtered_lines = bundler.process_lines(linesP)

# Draw them on canvas 
canvas = np.zeros(img.shape)
for i in range(0,len(filtered_lines)):
    l = filtered_lines[i][0]
    canvas = cv.line(canvas,(l[0], l[1]), (l[2], l[3]), (255,255,255),6, cv.LINE_AA)


In [ ]:
cv2_imshow(canvas)

In [ ]:
def findClosePointsV2(p1,p2,dist):
  hdist = np.abs(p1[1]-p2[1])
  if hdist<=dist :
    return True
  else:
    return False

def sort_contours(contoursList, x_axis_sort='LEFT_TO_RIGHT', y_axis_sort='TOP_TO_BOTTOM'):
    # initialize the reverse flag
    x_reverse = False
    y_reverse = False
    if x_axis_sort == 'RIGHT_TO_LEFT':
        x_reverse = True
    if y_axis_sort == 'BOTTOM_TO_TOP':
        y_reverse = True
    # Edit the contours 
    contours=[]
    for c in contoursList :
      c = np.asarray(c,dtype=np.int32).reshape(-1,2) 
      c = c.tolist()
      contours.append(c)
    boundingBoxes = [cv2.boundingRect(np.asarray(c)) for c in contours]
    # sorting on x-axis 
    sortedByX = zip(*sorted(zip(contours, boundingBoxes),
    key=lambda b:b[1][0], reverse=x_reverse))
    # sorting on y-axis 
    (contours, boundingBoxes) = zip(*sorted(zip(*sortedByX),
    key=lambda b:b[1][1], reverse=y_reverse))
    # return the list of sorted contours and bounding boxes
    return contours

def fuseScribbles(scribbles,patch,distanceThreshold=20):
    # Sort the scribbles 
    sortedScribbles = sort_contours(scribbles, x_axis_sort='RIGHT_TO_LEFT', y_axis_sort='TOP_TO_BOTTOM')
    
    # Collect all the startPoints , endPoints ( with their parent index )
    endPointDict = []
    startPointDict = []
    for i in range(0,len(sortedScribbles)):
      # Grab the start and end points.
      endPoint,startPoint = extremePoints(sortedScribbles[i])
      endPointDict.append(list(endPoint))
      startPointDict.append(list(startPoint))
    
    # Draw the connecting scribbles on the patch.
    gapCanvas = np.zeros(patch.shape)
    for i in range(0,len(startPointDict)):
      for j in range(0,len(endPointDict)):
          if i!=j:
            decision = findClosePointsV2(startPointDict[i],endPointDict[j],distanceThreshold)
            if decision: 
              gapCanvas = cv2.line(gapCanvas,startPointDict[i],endPointDict[j],color=(255,255,255),thickness=3)
            else:
              continue 
    
    # Return this canvas 
    return gapCanvas


In [ ]:
patch = np.uint8(canvas)
patch = cv2.cvtColor(patch,cv2.COLOR_BGR2GRAY)
contours, hierarchy = cv2.findContours(patch,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
gcanvas = fuseScribbles(contours,patch,distanceThreshold=10)
cv2_imshow(gcanvas)

In [ ]:
# gapCanvas = fuseScribbles(finscribbles,fcanvas,distanceThreshold=20)
netMap = cv2.bitwise_or(gcanvas,patch)
cv2_imshow(netMap)

# src = netMap
# # src = cv2.cvtColor(fcanvas,cv2.COLOR_BGR2GRAY)
# src[src<100]=0
# src[src>=100]=1
# src = np.uint8(src)
# scribbles = find_lines(src)

# # For visualisation 
# scanvas = np.zeros(src.shape)
# for c in scribbles:
#   c = np.asarray(c)
#   scanvas = cv2.polylines(scanvas,[c],False,(255,255,255),3)


In [ ]:

# New Utils 
def deformat(listofpoints):
    # Input : [[[x1,y1],[[x2,y2]],[[x3,y3]]....]
    # Output : [ [x1,y1], [x2,y2],[x3,y3]....]
    output = [ pt[0].tolist() for pt in listofpoints ]
    return output 

# Sorting 
def sort_contours(contoursList, x_axis_sort='LEFT_TO_RIGHT', y_axis_sort='TOP_TO_BOTTOM'):
    # initialize the reverse flag
    x_reverse = False
    y_reverse = False
    if x_axis_sort == 'RIGHT_TO_LEFT':
        x_reverse = True
    if y_axis_sort == 'BOTTOM_TO_TOP':
        y_reverse = True
    # Edit the contours 
    contours=[]
    for c in contoursList :
      c = np.asarray(c,dtype=np.int32).reshape(-1,2) 
      c = c.tolist()
      contours.append(c)
    boundingBoxes = [cv2.boundingRect(np.asarray(c)) for c in contours]
    # sorting on x-axis 
    sortedByX = zip(*sorted(zip(contours, boundingBoxes),
    key=lambda b:b[1][0], reverse=x_reverse))
    # sorting on y-axis 
    (contours, boundingBoxes) = zip(*sorted(zip(*sortedByX),
    key=lambda b:b[1][1], reverse=y_reverse))
    # return the list of sorted contours and bounding boxes
    return contours

# Supply the raw image here
def cleanImageFindContours(patch,threshold = 0.10):
  try:
    patch = np.uint8(patch)
    patch = cv2.cvtColor(patch,cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(patch,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if len(contours)<1:
      print('No contours in the raw image!')
      return patch
    # Else sort them 
    cntsSorted = sorted(contours, key=lambda x: cv2.contourArea(x),reverse=True)
    areaList = [cv2.contourArea(c) for c in cntsSorted]
    maxArea = max(areaList)
    sortedContours = [deformat(c) for c in cntsSorted if cv2.contourArea(c)>threshold*maxArea]
    return sortedContours

  except Exception as exp : 
    print('Error in figuring out the clean contours ')
    return None 
  

def deformatV2(listofpoints):
    # Input : [[[x1,y1],[[x2,y2]],[[x3,y3]]....]
    # Output : [ [x1,y1], [x2,y2],[x3,y3]....]
    output = [ pt[0] for pt in listofpoints ]
    return output 


def extremePoints(c):
    c = np.asarray(c).reshape(-1,1,2)
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    return extLeft,extRight

def find_lines(lines_mask,binaryThreshold=100):
    """
    Finds the longest central line for each connected component in the given binary mask.

    :param lines_mask: Binary mask of the detected line-areas
    :return: a list of Opencv-style polygonal lines (each contour encoded as [N,1,2] elements where each tuple is (x,y) )
    """
    
    # lines_mask = cv2.cvtColor(lines_mask,cv2.COLOR_BGR2GRAY)
    # lines_mask[lines_mask<binaryThreshold]=0
    # lines_mask[lines_mask>=binaryThreshold]=1
    # lines_mask = np.uint8(lines_mask)

    # Make sure one-pixel wide 8-connected mask
    lines_mask = skeletonize(lines_mask)

    class MakeLineMCP(MCP_Connect):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.connections = dict()
            self.scores = defaultdict(lambda: np.inf)

        def create_connection(self, id1, id2, pos1, pos2, cost1, cost2):
            k = (min(id1, id2), max(id1, id2))
            s = cost1 + cost2
            if self.scores[k] > s:
                self.connections[k] = (pos1, pos2, s)
                self.scores[k] = s

        def get_connections(self, subsample=5):
            results = dict()
            for k, (pos1, pos2, s) in self.connections.items():
                path = np.concatenate([self.traceback(pos1), self.traceback(pos2)[::-1]])
                results[k] = path[::subsample]
            return results

        def goal_reached(self, int_index, float_cumcost):
            if float_cumcost > 0:
                return 2
            else:
                return 0

    if np.sum(lines_mask) == 0:
        return []
    # Find extremities points
    end_points_candidates = np.stack(np.where((convolve2d(lines_mask, np.ones((3, 3)), mode='same') == 2) & lines_mask)).T
    connected_components = skimage_label(lines_mask, connectivity=2)
    # Group endpoint by connected components and keep only the two points furthest away
    d = defaultdict(list)
    for pt in end_points_candidates:
        d[connected_components[pt[0], pt[1]]].append(pt)
    end_points = []
    for pts in d.values():
        d = euclidean_distances(np.stack(pts), np.stack(pts))
        i, j = np.unravel_index(d.argmax(), d.shape)
        end_points.append(pts[i])
        end_points.append(pts[j])
    end_points = np.stack(end_points)

    mcp = MakeLineMCP(~lines_mask)
    mcp.find_costs(end_points)
    connections = mcp.get_connections()
    if not np.all(np.array(sorted([i for k in connections.keys() for i in k])) == np.arange(len(end_points))):
        print('Warning : find_lines seems weird')
    return [c[:, None, ::-1] for c in connections.values()]

def sort_contours(contoursList, x_axis_sort='LEFT_TO_RIGHT', y_axis_sort='TOP_TO_BOTTOM'):
    # initialize the reverse flag
    x_reverse = False
    y_reverse = False
    if x_axis_sort == 'RIGHT_TO_LEFT':
        x_reverse = True
    if y_axis_sort == 'BOTTOM_TO_TOP':
        y_reverse = True
    # Edit the contours 
    contours=[]
    for c in contoursList :
      c = np.asarray(c,dtype=np.int32).reshape(-1,2) 
      c = c.tolist()
      contours.append(c)
    boundingBoxes = [cv2.boundingRect(np.asarray(c)) for c in contours]
    # sorting on x-axis 
    sortedByX = zip(*sorted(zip(contours, boundingBoxes),
    key=lambda b:b[1][0], reverse=x_reverse))
    # sorting on y-axis 
    (contours, boundingBoxes) = zip(*sorted(zip(*sortedByX),
    key=lambda b:b[1][1], reverse=y_reverse))
    # return the list of sorted contours and bounding boxes
    return contours
    
def imageCombiner(imgs):
  imgs_comb = np.hstack([i for i in imgs])
  return imgs_comb



In [ ]:
def findClosePointsV2(p1,p2,dist):
  hdist = np.abs(p1[1]-p2[1])
  if hdist<=dist :
    return True
  else:
    return False

def fuseScribbles(scribbles,patch,distanceThreshold=20):
    # Sort the scribbles 
    sortedScribbles = sort_contours(scribbles, x_axis_sort='RIGHT_TO_LEFT', y_axis_sort='TOP_TO_BOTTOM')
    
    # Collect all the startPoints , endPoints ( with their parent index )
    endPointDict = []
    startPointDict = []
    for i in range(0,len(sortedScribbles)):
      # Grab the start and end points.
      endPoint,startPoint = extremePoints(sortedScribbles[i])
      endPointDict.append(list(endPoint))
      startPointDict.append(list(startPoint))
    
    # Draw the connecting scribbles on the patch.
    gapCanvas = np.zeros(patch.shape)
    for i in range(0,len(startPointDict)):
      for j in range(0,len(endPointDict)):
          if i!=j:
            decision = findClosePointsV2(startPointDict[i],endPointDict[j],distanceThreshold)
            if decision: 
              gapCanvas = cv2.line(gapCanvas,startPointDict[i],endPointDict[j],color=(255,255,255),thickness=3)
            else:
              continue 
    
    # Return this canvas 
    return gapCanvas


def localPatchProcessing(patch,distThreshold=20):
  try:
    # Apply Set of Erosion + Dilation to solve it 
    rectkernel = np.ones((1,15),np.uint8)
    erodeKernel =  np.ones((3,3),np.uint8)

    rawImage = copy.deepcopy(patch)

    rawImage = cv2.erode(rawImage,erodeKernel,iterations=2)
    rawImage = cv2.dilate(rawImage,rectkernel,iterations=6)
  
    # Cleaning it up a bit 
    try:
      finalContours = cleanImageFindContours(rawImage,threshold=0.1)
      assert len(finalContours)> 1 , 'No white patches !'
    except Exception as exp :
      return None

    # Plot all the filterd contours on canvs
    contourList = [] 
    finalCanvas = np.zeros((rawImage.shape[0],rawImage.shape[1]))
    for cnew in finalContours :
        contourList.append(cnew)
        points = np.array(cnew)
        # Fill the bbox 
        rect = cv2.minAreaRect(points)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        # Draw on the patch 
        # cv2.drawContours(im,[box],0,(0,0,255),2)
        finalCanvas = cv2.fillPoly(finalCanvas,pts=[box],color=(255,255,255))
        
    src = finalCanvas
    src[src<100]=0
    src[src>=100]=1
    src = np.uint8(src)
    scribbles = find_lines(src)
    

    # Parent Canvas 
    parentCanvas = np.zeros(rawImage.shape)
    for c in  scribbles:
      c = np.asarray(c)
      parentCanvas = cv2.polylines(parentCanvas,[c],False,(255,255,255),3)



    gapCanvas = fuseScribbles(scribbles,parentCanvas,distanceThreshold=20)
    netMap = cv2.bitwise_or(parentCanvas,gapCanvas)
    # print('NETMAP : ')
    # cv2_imshow(netMap)

    # # Apply find contours again 
    finalContours = cleanImageFindContours(netMap,threshold=0.1)

    # # Apply find contours on this one 
    fcanvas = np.zeros(netMap.shape)
    for c in finalContours:
      c = np.asarray(c)
      fcanvas = cv2.polylines(fcanvas,[c],False,(255,255,255),5)
    
    # Final set of scribbles
    src = fcanvas[:,:,0]
    # src = cv2.cvtColor(fcanvas,cv2.COLOR_BGR2GRAY)
    src[src<100]=0
    src[src>=100]=1
    src = np.uint8(src)
    finscribbles = find_lines(src)

    return finscribbles

  except Exception as exp :
    return None



def globalPostProcessing(localsmap,distanceThreshold=10):
  # Apply Horizontal Erosion with thickness 
  clean_image = localsmap.copy()
  rectkernel = np.ones((3,11),np.uint8)

  rawImage = copy.deepcopy(clean_image)
  rawImage = cv2.dilate(rawImage,rectkernel,iterations=7)

  # Detect the scribbles now and apply them again 
  # Final set of scribbles
  src = rawImage[:,:,0]
  # src = cv2.cvtColor(fcanvas,cv2.COLOR_BGR2GRAY)
  src[src<100]=0
  src[src>=100]=1
  src = np.uint8(src)

  # First set of scribbles ( better joined )
  finscribbles = find_lines(src)

  fcanvas = np.zeros(src.shape)
  for c in finscribbles:
    c = np.asarray(c)
    fcanvas = cv2.polylines(fcanvas,[c],False,(255,255,255),3)
  # cv2_imshow(fcanvas)

  # Fuse the remaining scribbles 
  gapCanvas = fuseScribbles(finscribbles,fcanvas,distanceThreshold=20)
  netMap = cv2.bitwise_or(fcanvas,gapCanvas)
  # cv2_imshow(netMap)

  src = netMap
  # src = cv2.cvtColor(fcanvas,cv2.COLOR_BGR2GRAY)
  src[src<100]=0
  src[src>=100]=1
  src = np.uint8(src)
  scribbles = find_lines(src)

  # For visualisation 
  scanvas = np.zeros(src.shape)
  for c in scribbles:
    c = np.asarray(c)
    scanvas = cv2.polylines(scanvas,[c],False,(255,255,255),3)
  
  return scribbles,scanvas


In [ ]:
def gluePipeline(smap,PDIM=512,OVERLAP=0.25):
  simg = copy.deepcopy(smap)
  emp = EMPatches()
  simg_patches, indices = emp.extract_patches(simg,patchsize=PDIM,overlap=OVERLAP)
  # Patchified Processing 
  clean_patches = []
  for i,sp in enumerate(simg_patches):
    pscribbles = localPatchProcessing(patch=sp)
    canvas = np.zeros(sp.shape)
    if pscribbles is not None :
      for cnew in pscribbles:
        canvas = cv2.polylines(canvas,[cnew],False,(255,255,255),3)
    clean_patches.append(canvas)
  # Merge back into clean canvas mode.
  clean_image = emp.merge_patches(clean_patches,indices)
  # Global Post Processing 
  scribbles,ncanvas = globalPostProcessing(localsmap=clean_image.copy(),distanceThreshold=10)   
  return scribbles,ncanvas


In [ ]:
scr,scanvas = gluePipeline(s,PDIM=512,OVERLAP=0.25)
cv2_imshow(scanvas)